In [ ]:
import os
import sys
import collections
import ipywidgets

import numpy
from matplotlib import pyplot
pyplot.style.use('{}/styles/'
                 'mesnardo.mplstyle'.format(os.environ['SCRIPTS']))
%matplotlib inline

sys.path.append(os.environ['SCRIPTS'])
from library.simulation import Simulation
from library.field import Field
import library.miscellaneous

In [ ]:
def get_vertical_gridline_values(field, x_target):
    indices = numpy.where(numpy.abs(field.x-x_target) <= 1.0E-06)[0]
    if indices.size == 0:
        i = numpy.where(field.x > x_target)[0][0]
        return ((abs(x[i]-x_target)*field.values[:, i-1]
               +abs(x[i-1]-x_target)*field.values[:, i])
               /abs(x[i]-x[i-1]))
    else:
        i = indices[0]
        return field.values[:, i]

In [ ]:
def get_horizontal_gridline_values(field, y_target):
    indices = numpy.where(numpy.abs(field.y-y_target) <= 1.0E-06)[0]
    if indices.size == 0:
        j = numpy.where(field.y > y_target)[0][0]
        return ((abs(y[j]-y_target)*field.values[j-1, :]
                +abs(y[j-1]-y_target)*field.values[j, :])
                /abs(y[j]-y[j-1]))
    else:
        j = indices[0]
        return field.values[j, :]

In [ ]:
def get_gridline_values(field, x_target=None, y_target=None):
    if x_target:
        return get_vertical_gridline_values(field, x_target)
    elif y_target:
        return get_horizontal_gridline_values(field, y_target)

In [ ]:
def plot_vertical_gridline_differences(simu1, simu2, 
                                      x_targets=[],
                                      time_steps=[],
                                      stride=1, log_yscale=False):
    simu1.read_grid()
    simu2.read_grid()
    fig, ax = pyplot.subplots(figsize=(6, 4))
    color_cycle = ax._get_lines.prop_cycler
    ax.grid(True, zorder=0)
    ax.set_xlabel('y-location')
    ax.set_ylabel('x-velocity differences\n along vertical gridline')
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    for time_step in time_steps:
        simu1.read_velocity(time_step)
        simu2.read_velocity(time_step)
        for x_target in x_targets:
            u1 = get_vertical_gridline_values(simu1.x_velocity, x_target)
            u2 = get_vertical_gridline_values(simu2.x_velocity, x_target)
            ax.plot(simu1.x_velocity.y[::stride], numpy.abs(u2-u1)[::stride],
                    label='nt={}, x={}'.format(time_step, x_target),
                    color=next(color_cycle)['color'], 
                    linestyle='-', linewidth=0,
                    marker='o', markersize=4,
                    zorder=10);
    ax.legend();
    if log_yscale:
        ax.set_yscale('log');

In [ ]:
def plot_horizontal_gridline_differences(simu1, simu2, 
                                         y_targets=[],
                                         time_steps=[],
                                         stride=1, log_yscale=False):
    simu1.read_grid()
    simu2.read_grid()
    fig, ax = pyplot.subplots(figsize=(6, 4))
    color_cycle = ax._get_lines.prop_cycler
    ax.grid(True, zorder=0)
    ax.set_xlabel('x-location')
    ax.set_ylabel('y-velocity differences\n along horizontal gridline')
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    for time_step in time_steps:
        simu1.read_velocity(time_step)
        simu2.read_velocity(time_step)
        for y_target in y_targets:
            v1 = get_horizontal_gridline_values(simu1.y_velocity, y_target)
            v2 = get_horizontal_gridline_values(simu2.y_velocity, y_target)
            ax.plot(simu1.y_velocity.x[::stride], numpy.abs(v2-v1)[::stride],
                    label='nt={}, y={}'.format(time_step, y_target),
                    color=next(color_cycle)['color'], 
                    linestyle='-', linewidth=0,
                    marker='o', markersize=4,
                    zorder=10);
    ax.legend();
    if log_yscale:
        ax.set_yscale('log');

In [ ]:
def plot_vertical_gridline_values(simulations, time_step, x_target, 
                                  xlimits=[None, None]):
    fig, ax = pyplot.subplots(figsize=(6, 4))
    color_cycle = ax._get_lines.prop_cycler
    ax.grid(True, zorder=0)
    ax.set_xlabel('y-location')
    ax.set_ylabel('x-velocity\n along vertical gridline')
    for simulation in simulations:
        simulation.read_grid()
        simulation.read_velocity(time_step)
        y = simulation.x_velocity.y
        u = get_vertical_gridline_values(simulation.x_velocity, x_target)
        if any(xlimits):
            mask = numpy.where(numpy.logical_and(y >= xlimits[0], y <= xlimits[1]))
            y = y[mask]
            u = u[mask]
        ax.plot(y, u,
                label=simulation.description,
                color=next(color_cycle)['color'],
                linestyle='-', marker='o', markersize=4,
                zorder=10)
    ax.legend();

In [ ]:
def plot_horizontal_gridline_values(simulations, time_step, y_target, 
                                    xlimits=[None, None]):
    fig, ax = pyplot.subplots(figsize=(6, 4))
    color_cycle = ax._get_lines.prop_cycler
    ax.grid(True, zorder=0)
    ax.set_xlabel('x-location')
    ax.set_ylabel('y-velocity\n along horizontal gridline')
    for simulation in simulations:
        simulation.read_grid()
        simulation.read_velocity(time_step)
        x = simulation.y_velocity.x
        v = get_horizontal_gridline_values(simulation.y_velocity, y_target)
        if any(xlimits):
            mask = numpy.where(numpy.logical_and(x >= xlimits[0], x <= xlimits[1]))
            x = x[mask]
            v = v[mask]
        ax.plot(x, v,
                label=simulation.description,
                color=next(color_cycle)['color'],
                linestyle='-', marker='o', markersize=4,
                zorder=10)
    ax.legend();

In [ ]:
cuibm_series_directory = ('{}/tmp/convergence_cuIBM_phantom'.format(os.environ['HOME']))
petibm_series_directory = ('{}/tmp/convergence_PetIBM_phantom'.format(os.environ['HOME']))

In [ ]:
gridline_size = 20

cuibm_directory = '{}/{}'.format(cuibm_series_directory, gridline_size)
cuibm = Simulation(software='cuibm', 
                   description='cuIBM', 
                   directory=cuibm_directory)
petibm_directory = '{}/{}'.format(petibm_series_directory, gridline_size)
petibm = Simulation(software='petibm', 
                    description='PetIBM',
                    directory=petibm_directory)

plot_vertical_gridline_values([petibm, cuibm], 500, 0.5)
plot_vertical_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.75, 0.85])
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5)
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.10, 0.2])

x_targets, y_targets = [0.5], [0.5]
time_steps = [500, 1000, 1500, 2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

x_targets, y_targets = [0.1, 0.5, 0.7, 0.9], [0.1, 0.5, 0.7, 0.9]
time_steps = [2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

In [ ]:
gridline_size = 60

cuibm_directory = '{}/{}'.format(cuibm_series_directory, gridline_size)
cuibm = Simulation(software='cuibm', 
                   description='cuIBM',
                   directory=cuibm_directory)
petibm_directory = '{}/{}'.format(petibm_series_directory, gridline_size)
petibm = Simulation(software='petibm', 
                    description='PetIBM',
                    directory=petibm_directory)

plot_vertical_gridline_values([petibm, cuibm], 500, 0.5)
plot_vertical_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.75, 0.85])
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5)
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.10, 0.2])

x_targets, y_targets = [0.5], [0.5]
time_steps = [500, 1000, 1500, 2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

x_targets, y_targets = [0.1, 0.5, 0.7, 0.9], [0.1, 0.5, 0.7, 0.9]
time_steps = [2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

In [ ]:
gridline_size = 180

cuibm_directory = '{}/{}'.format(cuibm_series_directory, gridline_size)
cuibm = Simulation(software='cuibm', 
                   description='cuIBM',
                   directory=cuibm_directory)
petibm_directory = '{}/{}'.format(petibm_series_directory, gridline_size)
petibm = Simulation(software='petibm', 
                    description='PetIBM',
                    directory=petibm_directory)

plot_vertical_gridline_values([petibm, cuibm], 500, 0.5)
plot_vertical_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.75, 0.85])
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5)
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.10, 0.2])

x_targets, y_targets = [0.5], [0.5]
time_steps = [500, 1000, 1500, 2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

x_targets, y_targets = [0.1, 0.5, 0.7, 0.9], [0.1, 0.5, 0.7, 0.9]
time_steps = [2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

In [ ]:
gridline_size = 540

cuibm_directory = '{}/{}'.format(cuibm_series_directory, gridline_size)
cuibm = Simulation(software='cuibm', 
                   description='cuIBM',
                   directory=cuibm_directory)
petibm_directory = '{}/{}'.format(petibm_series_directory, gridline_size)
petibm = Simulation(software='petibm', 
                    description='PetIBM',
                    directory=petibm_directory)

plot_vertical_gridline_values([petibm, cuibm], 500, 0.5)
plot_vertical_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.75, 0.85])
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5)
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.10, 0.20])

x_targets, y_targets = [0.5], [0.5]
time_steps = [500, 1000, 1500, 2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

x_targets, y_targets = [0.1, 0.5, 0.7, 0.9], [0.1, 0.5, 0.7, 0.9]
time_steps = [2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

In [ ]:
gridline_size = 1620

cuibm_directory = '{}/{}'.format(cuibm_series_directory, gridline_size)
cuibm = Simulation(software='cuibm', 
                   description='cuIBM',
                   directory=cuibm_directory)
petibm_directory = '{}/{}'.format(petibm_series_directory, gridline_size)
petibm = Simulation(software='petibm', 
                    description='PetIBM',
                    directory=petibm_directory)

plot_vertical_gridline_values([petibm, cuibm], 500, 0.5)
plot_vertical_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.75, 0.80])
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5)
plot_horizontal_gridline_values([petibm, cuibm], 500, 0.5, xlimits=[0.10, 0.15])

x_targets, y_targets = [0.5], [0.5]
time_steps = [500, 1000, 1500, 2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)

x_targets, y_targets = [0.1, 0.5, 0.7, 0.9], [0.1, 0.5, 0.7, 0.9]
time_steps = [2000]
plot_vertical_gridline_differences(petibm, cuibm, 
                                   x_targets=x_targets, 
                                   time_steps=time_steps,
                                   log_yscale=True)
plot_horizontal_gridline_differences(petibm, cuibm, 
                                     y_targets=y_targets, 
                                     time_steps=time_steps,
                                     log_yscale=True)